# Data Ingestion and Transformation using DataBricks (PySpark)

### 1. Importing Libraries

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [ ]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "",
"fs.azure.account.oauth2.client.secret": '',
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com//oauth2/token"}



### 2. Mounting Data from Azure Data lake 

In [ ]:
dbutils.fs.mount(
source = "abfss://tokyo-olypmics-data@tokyoolympicsdataudbhav.dfs.core.windows.net/", # contrainer@storageacc
mount_point = "/mnt/tokyoolymic",
extra_configs = configs)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-352692405588868>, line 1
----> 1 dbutils.fs.mount(
      2 source = "abfss://tokyo-olypmics-data@tokyoolympicsdataudbhav.dfs.core.windows.net/", # contrainer@storageacc
      3 mount_point = "/mnt/tokyoolymic",
      4 extra_configs = configs)

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o391.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/tokyoolymic; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/tokyoolymic
	at com.databricks.backend.daemon.data.cli

In [ ]:
display(dbutils.fs.ls("/mnt/tokyoolymic"))


path,name,size,modificationTime
dbfs:/mnt/tokyoolymic/raw-data/,raw-data/,0,1716659533000
dbfs:/mnt/tokyoolymic/transformed-data/,transformed-data/,0,1716659542000


### 3. Reading data into databricks from Azure Data lake

In [ ]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/Athlete.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/Coaches.csv")
entriesgender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/EntriesGender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/Medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/Teams.csv")
     

In [ ]:
athletes.printSchema()

root
 |-- PersonName: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)



In [ ]:
athletes.show()

+--------------------+--------------------+-------------------+
|          PersonName|             Country|         Discipline|
+--------------------+--------------------+-------------------+
|     AALERUD Katrine|              Norway|       Cycling Road|
|         ABAD Nestor|               Spain|Artistic Gymnastics|
|   ABAGNALE Giovanni|               Italy|             Rowing|
|      ABALDE Alberto|               Spain|         Basketball|
|       ABALDE Tamara|               Spain|         Basketball|
|           ABALO Luc|              France|           Handball|
|        ABAROA Cesar|               Chile|             Rowing|
|       ABASS Abobakr|               Sudan|           Swimming|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|
|       ABBASOV Islam|          Azerbaijan|          Wrestling|
|        ABBINGH Lois|         Netherlands|           Handball|
|         ABBOT Emily|           Australia|Rhythmic Gymnastics|
|       ABBOTT Monica|United States of .

In [ ]:
coaches.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
coaches.show()

+--------------------+--------------------+-----------------+--------+
|                Name|             Country|       Discipline|   Event|
+--------------------+--------------------+-----------------+--------+
|     ABDELMAGID Wael|               Egypt|         Football|    NULL|
|           ABE Junya|               Japan|       Volleyball|    NULL|
|       ABE Katsuhiko|               Japan|       Basketball|    NULL|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|    NULL|
|          AGEBA Yuya|               Japan|       Volleyball|    NULL|
|AIKMAN Siegfried ...|               Japan|           Hockey|     Men|
|       AL SAADI Kais|             Germany|           Hockey|     Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Softball|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|     Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|   Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|    NULL|
|     

In [ ]:
entriesgender.printSchema()

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [ ]:
entriesgender.show()

+--------------------+------+----+-----+
|          Discipline|Female|Male|Total|
+--------------------+------+----+-----+
|      3x3 Basketball|    32|  32|   64|
|             Archery|    64|  64|  128|
| Artistic Gymnastics|    98|  98|  196|
|   Artistic Swimming|   105|   0|  105|
|           Athletics|   969|1072| 2041|
|           Badminton|    86|  87|  173|
|   Baseball/Softball|    90| 144|  234|
|          Basketball|   144| 144|  288|
|    Beach Volleyball|    48|  48|   96|
|              Boxing|   102| 187|  289|
|        Canoe Slalom|    41|  41|   82|
|        Canoe Sprint|   123| 126|  249|
|Cycling BMX Frees...|    10|   9|   19|
|  Cycling BMX Racing|    24|  24|   48|
|Cycling Mountain ...|    38|  38|   76|
|        Cycling Road|    70| 131|  201|
|       Cycling Track|    90|  99|  189|
|              Diving|    72|  71|  143|
|          Equestrian|    73| 125|  198|
|             Fencing|   107| 108|  215|
+--------------------+------+----+-----+
only showing top

In [ ]:
medals.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Team_Country: string (nullable = true)
 |-- Gold: integer (nullable = true)
 |-- Silver: integer (nullable = true)
 |-- Bronze: integer (nullable = true)
 |-- Total: integer (nullable = true)
 |-- Rank by Total: integer (nullable = true)



In [ ]:
medals.show()

+----+--------------------+----+------+------+-----+-------------+
|Rank|        Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+--------------------+----+------+------+-----+-------------+
|   1|United States of ...|  39|    41|    33|  113|            1|
|   2|People's Republic...|  38|    32|    18|   88|            2|
|   3|               Japan|  27|    14|    17|   58|            5|
|   4|       Great Britain|  22|    21|    22|   65|            4|
|   5|                 ROC|  20|    28|    23|   71|            3|
|   6|           Australia|  17|     7|    22|   46|            6|
|   7|         Netherlands|  10|    12|    14|   36|            9|
|   8|              France|  10|    12|    11|   33|           10|
|   9|             Germany|  10|    11|    16|   37|            8|
|  10|               Italy|  10|    10|    20|   40|            7|
|  11|              Canada|   7|     6|    11|   24|           11|
|  12|              Brazil|   7|     6|     8|   21|          

In [ ]:
teams.printSchema()

root
 |-- TeamName: string (nullable = true)
 |-- Discipline: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Event: string (nullable = true)



In [ ]:
teams.show()

+-------------+--------------+--------------------+------------+
|     TeamName|    Discipline|             Country|       Event|
+-------------+--------------+--------------------+------------+
|      Belgium|3x3 Basketball|             Belgium|         Men|
|        China|3x3 Basketball|People's Republic...|         Men|
|        China|3x3 Basketball|People's Republic...|       Women|
|       France|3x3 Basketball|              France|       Women|
|        Italy|3x3 Basketball|               Italy|       Women|
|        Japan|3x3 Basketball|               Japan|         Men|
|        Japan|3x3 Basketball|               Japan|       Women|
|       Latvia|3x3 Basketball|              Latvia|         Men|
|     Mongolia|3x3 Basketball|            Mongolia|       Women|
|  Netherlands|3x3 Basketball|         Netherlands|         Men|
|       Poland|3x3 Basketball|              Poland|         Men|
|          ROC|3x3 Basketball|                 ROC|         Men|
|          ROC|3x3 Basket

### 4. Cleaning and tranformations
##### Finding Nulls in the data

In [ ]:
from pyspark.sql.functions import col, sum

athletes_nulls = athletes.select([sum(col(c).isNull().cast("int")).alias(c) for c in athletes.columns])
athletes_nulls.show()


+----------+-------+----------+
|PersonName|Country|Discipline|
+----------+-------+----------+
|         0|      0|         0|
+----------+-------+----------+



In [ ]:
coaches_nulls = coaches.select([sum(col(c).isNull().cast("int")).alias(c) for c in coaches.columns])
coaches_nulls.show()


+----+-------+----------+-----+
|Name|Country|Discipline|Event|
+----+-------+----------+-----+
|   0|      0|         0|  145|
+----+-------+----------+-----+



In [ ]:
entriesgender_nulls = entriesgender.select([sum(col(c).isNull().cast("int")).alias(c) for c in entriesgender.columns])
entriesgender_nulls.show()


+----------+------+----+-----+
|Discipline|Female|Male|Total|
+----------+------+----+-----+
|         0|     0|   0|    0|
+----------+------+----+-----+



In [ ]:
medals_nulls = medals.select([sum(col(c).isNull().cast("int")).alias(c) for c in medals.columns])
medals_nulls.show()


+----+------------+----+------+------+-----+-------------+
|Rank|Team_Country|Gold|Silver|Bronze|Total|Rank by Total|
+----+------------+----+------+------+-----+-------------+
|   0|           0|   0|     0|     0|    0|            0|
+----+------------+----+------+------+-----+-------------+



In [ ]:
teams_nulls = teams.select([sum(col(c).isNull().cast("int")).alias(c) for c in teams.columns])
teams_nulls.show()


+--------+----------+-------+-----+
|TeamName|Discipline|Country|Event|
+--------+----------+-------+-----+
|       0|         0|      0|    0|
+--------+----------+-------+-----+



##### We saw that Coaches table has 145 nulls in Event column. Let's impute it as 'Unknown'

In [ ]:
from pyspark.sql.functions import col, when, lit
# Impute null values in the 'Event' column with 'Unknown' only for rows with invalid events
coaches = coaches.withColumn('Event', when(~col('Event').isin('Men', 'Women'), lit('Unknown')).otherwise(col('Event')))
coaches = coaches.fillna({'Event': 'Unknown'})
# Show the result to verify
coaches.show()

+--------------------+--------------------+-----------------+-------+
|                Name|             Country|       Discipline|  Event|
+--------------------+--------------------+-----------------+-------+
|     ABDELMAGID Wael|               Egypt|         Football|Unknown|
|           ABE Junya|               Japan|       Volleyball|Unknown|
|       ABE Katsuhiko|               Japan|       Basketball|Unknown|
|        ADAMA Cherif|       C�te d'Ivoire|         Football|Unknown|
|          AGEBA Yuya|               Japan|       Volleyball|Unknown|
|AIKMAN Siegfried ...|               Japan|           Hockey|    Men|
|       AL SAADI Kais|             Germany|           Hockey|    Men|
|       ALAMEDA Lonni|              Canada|Baseball/Softball|Unknown|
|     ALEKNO Vladimir|Islamic Republic ...|       Volleyball|    Men|
|     ALEKSEEV Alexey|                 ROC|         Handball|  Women|
|ALLER CARBALLO Ma...|               Spain|       Basketball|Unknown|
|       ALSHEHRI Saa

##### Splitting the Name into first and last Name

In [ ]:
from pyspark.sql.functions import split, col
athletes = athletes.withColumn('FirstName', split(col('PersonName'), ' ')[0]) \
                                .withColumn('LastName', split(col('PersonName'), ' ')[1])

athletes.show()

+--------------------+--------------------+-------------------+-----------+--------+
|          PersonName|             Country|         Discipline|  FirstName|LastName|
+--------------------+--------------------+-------------------+-----------+--------+
|     AALERUD Katrine|              Norway|       Cycling Road|    AALERUD| Katrine|
|         ABAD Nestor|               Spain|Artistic Gymnastics|       ABAD|  Nestor|
|   ABAGNALE Giovanni|               Italy|             Rowing|   ABAGNALE|Giovanni|
|      ABALDE Alberto|               Spain|         Basketball|     ABALDE| Alberto|
|       ABALDE Tamara|               Spain|         Basketball|     ABALDE|  Tamara|
|           ABALO Luc|              France|           Handball|      ABALO|     Luc|
|        ABAROA Cesar|               Chile|             Rowing|     ABAROA|   Cesar|
|       ABASS Abobakr|               Sudan|           Swimming|      ABASS| Abobakr|
|    ABBASALI Hamideh|Islamic Republic ...|             Karate|  

##### Convert String Columns to Proper Case:

In [ ]:
from pyspark.sql.functions import initcap

# Convert string columns to proper case
athletes_transformed = athletes.withColumn('Country', initcap(col('Country'))) \
                               .withColumn('Discipline', initcap(col('Discipline')))

coaches_transformed = coaches.withColumn('Country', initcap(col('Country'))) \
                             .withColumn('Discipline', initcap(col('Discipline'))) \
                             .withColumn('Event', initcap(col('Event')))


In [ ]:
athletes.show(5)

+-----------------+-------+-------------------+---------+--------+
|       PersonName|Country|         Discipline|FirstName|LastName|
+-----------------+-------+-------------------+---------+--------+
|  AALERUD Katrine| Norway|       Cycling Road|  AALERUD| Katrine|
|      ABAD Nestor|  Spain|Artistic Gymnastics|     ABAD|  Nestor|
|ABAGNALE Giovanni|  Italy|             Rowing| ABAGNALE|Giovanni|
|   ABALDE Alberto|  Spain|         Basketball|   ABALDE| Alberto|
|    ABALDE Tamara|  Spain|         Basketball|   ABALDE|  Tamara|
+-----------------+-------+-------------------+---------+--------+
only showing top 5 rows



In [ ]:
coaches.show(5)

+---------------+-------------+----------+-------+
|           Name|      Country|Discipline|  Event|
+---------------+-------------+----------+-------+
|ABDELMAGID Wael|        Egypt|  Football|Unknown|
|      ABE Junya|        Japan|Volleyball|Unknown|
|  ABE Katsuhiko|        Japan|Basketball|Unknown|
|   ADAMA Cherif|C�te d'Ivoire|  Football|Unknown|
|     AGEBA Yuya|        Japan|Volleyball|Unknown|
+---------------+-------------+----------+-------+
only showing top 5 rows



### 5. Loading into Datalake

In [ ]:
athletes.repartition(1).write.mode("overwrite").option("header",'true').csv("/mnt/tokyoolymic/transformed-data/athletes")
coaches.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/coaches")
entriesgender.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/entriesgender")
medals.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/medals")
teams.repartition(1).write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/teams")